In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets
digits = datasets.load_digits()

images_number = 5
images_and_labels = list(zip(digits.images, digits.target))

for i, (image, label) in enumerate(images_and_labels[:images_number]):
    plt.subplot(2, images_number, i + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('%i' % label)

In [ ]:
print(digits.data[0])
print(digits.target[0])
samples_count = len(digits.images)
print("Number of samples: " + str(samples_count))

In [ ]:
train_data = digits.data[: int(0.7*samples_count)]
train_labels = digits.target[: int(0.7*samples_count)]

test_data = digits.data[int(0.7*samples_count): int(0.9*samples_count)]
test_labels = digits.target[int(0.7*samples_count): int(0.9*samples_count)]

score_data = digits.data[int(0.9*samples_count): ]
score_labels = digits.target[int(0.9*samples_count): ]

print("Number of training records: " + str(len(train_data)))
print("Number of testing records : " + str(len(test_data)))
print("Number of scoring records : " + str(len(score_data)))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import svm, metrics

scaler = preprocessing.StandardScaler()
clf = svm.SVC(kernel='rbf')
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])

model = pipeline.fit(train_data, train_labels)

predicted = model.predict(test_data)
print("Evaluation report: \n\n%s" % metrics.classification_report(test_labels, predicted))

In [ ]:
# このパラメータは自分の環境のものに置き換えて下さい
wml_credentials={
        "url": "https://ibm-watson-ml.mybluemix.net",
        "access_key": "xxxx",
        "username": "xxxx",
        "password": "xxxx",
        "instance_id": "xxxx"
}

In [ ]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps, MetaNames

ml_repository_client = MLRepositoryClient(wml_credentials['url'])
ml_repository_client.authorize(wml_credentials['username'], wml_credentials['password'])

# このパラメータは正しいものに修正して下さい
props = MetaProps({MetaNames.AUTHOR_NAME:"name", MetaNames.AUTHOR_EMAIL:"xxx@yyy.zzz"})
model_artifact = MLRepositoryArtifact(model, name="Hand-written digits recognition", meta_props=props)

In [ ]:
saved_model = ml_repository_client.models.save(model_artifact)

In [ ]:
print(saved_model.meta.available_props())
print("modelType: " + saved_model.meta.prop("modelType"))
print("runtime: " + saved_model.meta.prop("runtime"))
print("creationTime: " + str(saved_model.meta.prop("creationTime")))
print("modelVersionHref: " + saved_model.meta.prop("modelVersionHref"))

In [ ]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)
print(loadedModelArtifact.name)

predictions = loadedModelArtifact.model_instance().predict(score_data)
print(predictions)

In [ ]:
# 必要ライブラリのロード
import urllib3
import requests
import json
# Basic認証用ヘッダの生成
auth = '{username}:{password}'.format(
    username=wml_credentials['username'], password=wml_credentials['password'])
header_basic_auth = urllib3.util.make_headers(basic_auth=auth)
url = '{}/v3/identity/token'.format(wml_credentials['url'])
# Tokenの取得
mltoken =  json.loads( requests.get(url, headers=header_basic_auth).text )['token']

In [ ]:
# インスタンス取得用エンドポイントURL
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
# Tokenを使ったヘッダの生成
header_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
# インスタンス情報取得
instance = json.loads( requests.get(endpoint_instance, headers=header_token).text )
# 登録済みモデル一覧取得用URL
endpoint_published_models = instance['entity']['published_models']['url']

In [ ]:
# 登録済みモデル一覧の取得
published_models = json.loads( requests.get(endpoint_published_models, headers=header_token).text )
# uidをキーにWebサービス化を行いたいモデルを選択し、そのモデルのdeploy用URLを取得する
[endpoint_deployments] = [ 
    x['entity']['deployments']['url'] for x in published_models['resources'] 
    if x['metadata']['guid'] == saved_model.uid ]

In [ ]:
# Webサービス登録用パラメータ設定
payload_online = {
    "name": "Hand written digits recognition", 
    "description": "Hand Written Digits Deployment", 
    "type": "online"}
# Webサービス登録
online = json.loads( requests.post(endpoint_deployments, json=payload_online, headers=header_token).text )
# 登録したWebサービスのURL
scoring_url = online['entity']['scoring_url']

In [ ]:
# Webサービス呼出し用パラメータの設定
payload_scoring = {"values": [list(data) for data in score_data]}
# Webサービスの呼出し
scoring = json.loads( requests.post(scoring_url, json=payload_scoring, headers=header_token).text )

In [ ]:
# 呼出し結果の取得
predict_list = np.array([item[0] for item in scoring['values']])
# 予測結果の表示
print(predict_list)
# 正解データの表示
print(score_labels)
# 正解率の表示
print("Evaluation report: \n\n%s" % metrics.classification_report(score_labels, predict_list))